In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from tqdm.notebook import tqdm

from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi

from qc_grader.challenges.spring_2022.helpers import generate_XX, generate_YY, generate_disordered_tb_instruction

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
t = Parameter('t')

In [ ]:
XX = generate_XX(t)
YY = generate_YY(t)

# 5 격자점 밀접 결합 격자

이 연습는 5개의 격자점이 있는 밀접 결합 격자를 시뮬레이션 해보도록 하겠습니다. 모든 격자점이 동일한 에너지 값을 지니는 밀접 결합 해밀토니안을 살펴보는 것부터 시작해 보겠습니다. 지난 연습에서 배운내용을 환기하여 5 격자점 밀접 결합 격자의 해밀토니언을 작성해 봅시다.

$$H_{\rm tb}/\hbar = J \sum^{3} _{i=0} (X_i X_{i+1} + Y_i Y_{i+1})$$

이번 도전을 위해, 일반성을 잃지 않고, $\hbar=1$ 및 $J=1$로 설정합니다.

## 1. 입자 수송

밀접 결합 시스템에서 입자의 전파(propagation)는 연속 시간 양자 무작위 걷기(Quantum random walk)로 설명할 수 있습니다. 양자 무작위 걷기는 고전적인 무작위 걷기의 양자 역학적인 대응입니다. 무작위 걷기는 무작위로 움직이는 입자가 시작점에서 멀어지는 과정입니다. 고전적 무작위 걷기에서 시간 $t$에 시작점에서 $r$만큼 떨어진 위치에서 입자를 찾을 확률은 다음과 같이 가우스 분포를 따릅니다. $$ p_{\text{classical}}(r,t) \propto e^{-|r|^2/t} $$. 이 확률 분포의 표준 편차는 시간이 지남에 따라 제곱근으로 증가합니다: $\sigma_{\text{classical}}= \sqrt{t}$. 단일 입자에서 발생하는 중첩 및 간섭과 같은 양자 속성으로 인해 고전적 무작위 걷기와 양자 무작위 걷기 간의 질적인 차이가 발생합니다. 양자 무작위 걷기에서 시작점에서 $r$만큼 떨어진 위치에 있는 입자를 찾을 확률은 [제1종 베셀 함수를](https://en.wikipedia.org/wiki/Bessel_function) 따릅니다: $$ p_{\text{quantum}}(r,t) \propto |J_r( 2t)|^2 $$. 이 확률 분포의 표준 편차는 시간에 따라 선형적으로 증가합니다: $\sigma_{\text{quantum}} = t$.

In [ ]:
from scipy.special import jv

t_test = 20
r = np.linspace(-50,50,101)
gaussian = np.exp(-r**2/t_test) / np.sum(np.exp(-r**2/t_test))
bessel = np.abs(jv(r,2*t_test))**2 / np.sum(np.abs(jv(r,2*t_test))**2)

plt.plot(r, gaussian, label=r'Gaussian function')
plt.plot(r, bessel, label=r'Bessel function')

plt.xlabel('Position')
plt.ylabel('Probability density')
plt.legend()
plt.show()

격자 전체에 걸쳐 발생하는 입자의 공간 전파는 초기 위치와 관련하여 제곱 평균 변위(Mean square displacement) $⟨x^2⟩= \sum_i p_i x^2_i$로 양을 정의할 수 있으며, 여기서 $p_i$는 격자점 $i$에서 입자를 찾을 확률을 나타탭니다. 고전적인 무작위 걷기는 $⟨x^2⟩ \propto t$로 시간에 따라 확산 전파되는 반면 QRW는 평균 제곱 변위 $⟨x^2⟩ \propto t^2$로 탄도 전파(ballistic propagation)의 특성을 보입니다.

고전적 무작위 걷기와 비교하여 양자 무작위 걷기의 이차(quadratic) 속도 향상은 Grover 검색 알고리즘이 고전적 검색에 비해 이차 속도 향상을 갖는 것과 유사합니다.

<div class="alert alert-block alert-danger">

<b>도전 질문 2a</b>
밀접 결합 해밀토니언의 trotterized 시간적 변화의 양자 계산을 위해 XX 및 YY 게이트를 회로에 추가해 봅시다.
</div>

In [ ]:
num_qubits = 5 ## DO NOT EDIT

Trot_tb_qr = QuantumRegister(num_qubits)
Trot_tb_qc = QuantumCircuit(Trot_tb_qr, name='Trot')

###EDIT CODE BELOW



###DO NOT EDIT BELOW

Trot_tb_gate = Trot_tb_qc.to_instruction()

Trot_tb_qc.draw()

In [ ]:
## Grade and submit your solution
from qc_grader.challenges.spring_2022 import grade_ex2a

grade_ex2a(Trot_tb_qc)


<div class="alert alert-block alert-danger">

<b data-md-type="raw_html">도전 질문 2b</b>
이어서, 들뜸 형태의 입자를 추가해 봅시다. 해밀토니언에 시간적 변화를 일으키기 전, 큐비트에  𝑋 게이트를 적용해 |0⟩에서 |1⟩으로 상태를 뒤집어서 준비합니다.
</div>

In [ ]:
delta_t=0.15 # DO NOT EDIT
time_steps=np.arange(1,20,1) # DO NOT EDIT

circuits=[]

for n_steps in time_steps:
    
    qr = QuantumRegister(num_qubits)
    cr = ClassicalRegister(num_qubits)
    qc = QuantumCircuit(qr,cr)

    ###EDIT CODE BELOW
    
    ###DO NOT EDIT BELOW
    
    for _ in range(n_steps):
        qc.append(Trot_tb_gate, [i for i in range(num_qubits)])
        
    qc = qc.bind_parameters({t: delta_t})
    
    circuits.append(qc)

매 시간 단계마다 각각 큐비트에서 들뜬 상태를 찾을 확률을 추적하여 입자의 위치를 추적해 봅시다. 

<div class="alert alert-block alert-danger">
<b>도전 질문 2c</b>
먼저, statevector_simulator의 출력 상태를 사용해 매 시간단계에서 각각의 큐비트가 $|1\rangle$의 상태에 있을 확률을 추출합니다.
</div>

In [ ]:
from qiskit import transpile

# Use Aer's qasm_simulator
from qiskit import Aer

# Run the quantum circuit on a statevector simulator backend
backend_sim = Aer.get_backend('statevector_simulator')

probability_density=[]
for circ in tqdm(circuits):

    transpiled_circ=transpile(circ, backend_sim, optimization_level=3)

    job_sim = backend_sim.run(transpiled_circ)

    # Grab the results from the job.
    result_sim = job_sim.result()
    outputstate = result_sim.get_statevector(transpiled_circ, decimals=5)
    
    ps=[]
    
    ###EDIT CODE BELOW (Extract the probability of finding the excitation on each qubit)
    
    ###DO NOT EDIT BELOW
    
    probability_density.append(ps)
    
probability_density=np.array(probability_density)

In [ ]:
plt.figure(figsize=(3,5), facecolor='white')
plt.pcolormesh(np.arange(0,num_qubits,1), time_steps*delta_t, probability_density)
plt.xlabel('Qubit index')
plt.ylabel('Time (1/J)')

In [ ]:
## Grade and submit your solution
from qc_grader.challenges.spring_2022 import grade_ex2b

grade_ex2b(probability_density)


<div class="alert alert-block alert-info">
<b>하드웨어 챌린지 I</b>
실제 하드웨어에서 양자 무작위 걷기를 위한 회로를 실행하고 각 큐비트에서 들뜬 상태를 찾을 확률을 추출합니다.
</div>

In [ ]:
from qiskit.tools.monitor import job_monitor

provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibm_perth')

In [ ]:
initial_layout=[0, 1, 3, 5, 6]

hardware_transpiled_circuits=[]
for circ in circuits:
    
    hardware_circ=circ
    hardware_circ.measure(range(num_qubits), range(num_qubits))
    hardware_transpiled_circuits.append(transpile(hardware_circ, backend, initial_layout=initial_layout, optimization_level=3))

In [ ]:
shots=1024
job = execute(hardware_transpiled_circuits, backend=backend, shots=shots)
job_monitor(job)
print('Job ID', job.job_id())

In [ ]:
experiment_results=job.result()

probability_density_exp=[]
for output in experiment_results.get_counts():
    ps=[]
    
    ###EDIT CODE BELOW (Extract the probability of finding the excitation on each qubit)
    
    ###DO NOT EDIT BELOW
    
    probability_density_exp.append(ps)

plt.figure(figsize=(3,5), facecolor='white')
plt.pcolormesh(np.arange(-.5,num_qubits,1), time_steps*delta_t, probability_density_exp)
plt.xlabel('Qubit index')
plt.ylabel('Time (1/J)')

## 2. 앤더슨 국소화(Anderson localization)

격자점 에너지에 무질서가 유입되면 앤더슨 국소화로 이어져 입자의 전파가 크게 영향을 받게 됩니다. 격자의 비균질성은 산란을 일으키고 입자 전파를 억제하는 특성을 지니는 양자 간섭을 일으키는데 이는 국소화가 발생할때의 특징입니다. 국소화된 입자의 파동 함수는 입자의 초기 위치에서 멀어질수록 빠르게 감소하므로 입자를 격자의 작은 영역에 머물도록 제한하게 됩니다.

Phillip Anderson은 1958년에 무질서로인한 산란이 수송을 완전히 멈출 수 있다고 처음으로 제안했습니다[1]. Anderson의 발견 이전에 과학자들은 전자를 점과 같은 입자로 취급하여, 결정의 무질서를 무작위로 전자를 산란시키는 섭동(perturbation)으로 모델링했습니다. 이 논리는 브라운 운동과 같은 매체에서의 운송에 대한 이론으로 이어졌으며 이 이론은 옴의 법칙의 기초가 되었습니다. 그러나 Anderson은 주기적인 결정에서 전자의 파동함수의 진화에 무질서가 미치는 영향을 재검토했습니다[1]. Anderson은 양자 영역에서 문제를 분석하여, 전자의 파동 특성을 근본적으로 고려함으로서 전자의 고전적인 확산 움직임이, 광범위한 조건에서 전자 파동 함수가 기하급수적으로 국지화됨에 따라 더이상 들어맞지 않게 된다는 것을 발견했습니다. 따라서, 전자가 초기에 한 원자에 존재한다면, 파동함수는 시간이 지나도 더 이상 전체 결정을 덮을 정도로 확장되지 않으며, 오히려 초기 위치 주변에 국한된 상태로 유지됩니다. 따라서 물질은 전하의 전도를 멈추고 결국 절연체가 됩니다. 이 국부화 현상은 격자 무질서에 의해 촉발되는 전자의 다중 산란에 의한 서로 다른 경로 간의 간섭이 만들어내는 직접적인 결과입니다.


<div></div>
<img src="attachment:Anderson_localization.jpg" width="600">[2]

밀접 결합 해밀토니언을 사용하여 앤더슨 국소화를 연구할 수 있습니다. 격자점 에너지에 무질서가 존재할 때 해밀토니언은 다음 형식을 취합니다. $$H_{\rm tb}/\hbar = \sum^{3}_{i=0} (X_i X_{i+1} + Y_i Y_{i+1}) + \sum_i \epsilon_i Z_i $$

$\epsilon_i$ 값을 변경하면 앤더슨 국소화로 이어지는 Aubry-Andre(AA) 준결정(quasicrystal)을 생성할 수 있고 이 과정을 통해 무질서를 모방할 수 있습니다. 이 격자 모델에서 $\epsilon_i=W \rm{cos}(2\pi\beta i)$이고 여기서 $W$는 무질서 강도이며 $\beta$는 준결정의 주기성을 결정합니다.

[1] https://physicstoday.scitation.org/doi/10.1063/1.3206091?feed=most-cited <br><br>[2] https://www.nature.com/articles/nphoton.2013.30

In [ ]:
beta=(np.sqrt(5)-1)/2 # DO NOT EDIT
AA_patern=np.cos(2*np.pi*beta*np.arange(num_qubits)) # DO NOT EDIT

plt.plot(np.linspace(-0.5,4.5,101), np.cos(2*np.pi*beta*np.linspace(-0.5,4.5,101)), '--')
plt.plot(np.arange(num_qubits), AA_patern, 'o', label=r'$\epsilon_i$')
plt.xlabel('Position')
plt.ylabel('Energy')
plt.legend()
plt.title('Aubry-Andre potential')
plt.show()

<div class="alert alert-block alert-danger">
<b>도전 질문 2d</b>
앤더슨 국소화를 시뮬레이션하는 데 사용되는 임의의 상태를 포함하도록 각 밀접 결합 trotter 단계를 수정합니다.
</div>

In [ ]:
Trot_qr_disorder = QuantumRegister(num_qubits)
Trot_qc_disorder = QuantumCircuit(Trot_qr_disorder, name='Trot disorder')

Trot_qc_disorder.append(Trot_tb_gate,[0,1,2,3,4])
deltas=[Parameter('delta_{:d}'.format(idx)) for idx in range(num_qubits)]

###EDIT CODE BELOW (add a parametric disorder to each qubit)


###DO NOT EDIT BELOW
    
# Convert custom quantum circuit into a gate
Trot_disorder_gate = Trot_qc_disorder.to_instruction()

Trot_qc_disorder.draw()

In [ ]:
delta_t=0.15
time_steps=np.arange(1,20,1)

W=2 # DO NOT EDIT

disorders=W*AA_patern # DO NOT EDIT

disorder_circuits=[]

for n_steps in time_steps:

    qr = QuantumRegister(num_qubits)
    cr = ClassicalRegister(num_qubits)
    qc = QuantumCircuit(qr, cr)

    qc.x(0)
    
    for _ in range(n_steps):
        qc.append(Trot_disorder_gate, [i for i in range(num_qubits)])

    qc = qc.bind_parameters({t: delta_t})
    qc = qc.bind_parameters({deltas[idx]: disorders[idx] for idx in range(num_qubits)})

    disorder_circuits.append(qc)

<div class="alert alert-block alert-danger">
<b>도전 질문 2e</b>
마지막으로, 각 큐비트에서 들뜸 상태를 찾을 확률을 추출합니다.
</div>

In [ ]:
from qiskit import transpile

# Use Aer's qasm_simulator
from qiskit import Aer

# Run the quantum circuit on a statevector simulator backend
backend_sim = Aer.get_backend('statevector_simulator')

probability_density_localization=[]
for circ in tqdm(disorder_circuits):

    transpiled_circ=transpile(circ, backend_sim, optimization_level=3)

    job_sim = backend_sim.run(transpiled_circ)

    # Grab the results from the job.
    result_sim = job_sim.result()
    outputstate = result_sim.get_statevector(transpiled_circ, decimals=5)
    
    ps=[]
    
    ###EDIT CODE BELOW (Extract the probability of finding the excitation on each qubit)
    
    ###DO NOT EDIT BELOW
    
    probability_density_localization.append(ps)
    
probability_density_localization=np.array(probability_density_localization)

In [ ]:
plt.figure(figsize=(3,5), facecolor='white')
plt.pcolormesh(np.arange(0,num_qubits,1), time_steps*delta_t ,probability_density_localization)
plt.xlabel('Qubit index')
plt.ylabel('Time (1/J)')

In [ ]:
## Grade and submit your solution
from qc_grader.challenges.spring_2022 import grade_ex2c

grade_ex2c(probability_density_localization)


<div class="alert alert-block alert-info">
<b>하드웨어 도전 II</b>
실제 하드웨어에서 무질서가 있는 양자 무작위 걷기에 대한 회로를 실행하고 각 큐비트에서 들뜸 상태를 찾을 확률을 추출합니다.
</div>

In [ ]:
from qiskit.tools.monitor import job_monitor

provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibm_perth')

In [ ]:
initial_layout=[0, 1, 3, 5, 6]

hardware_transpiled_circuits_disordered=[]
for circ in disorder_circuits:
    
    hardware_circ=circ
    hardware_circ.measure(range(num_qubits), range(num_qubits))
    hardware_transpiled_circuits_disordered.append(transpile(hardware_circ, backend, initial_layout=initial_layout, optimization_level=3))

In [ ]:
shots=1024
job_disorder = execute(hardware_transpiled_circuits_disordered, backend=backend, shots=shots)
job_monitor(job_disorder)
print('Job ID', job_disorder.job_id())

In [ ]:
disorder_experiment_results=job_disorder.result()

disorder_probability_density_exp=[]
for output in disorder_experiment_results.get_counts():
    ps=[]
    
    ###EDIT CODE BELOW (Extract the probability of finding the excitation on each qubit)
    
    ###DO NOT EDIT BELOW
    
    disorder_probability_density_exp.append(ps)
    
plt.figure(figsize=(3,5), facecolor='white')
plt.pcolormesh(np.arange(-.5,num_qubits,1), time_steps*delta_t, disorder_probability_density_exp)
plt.xlabel('Qubit index')
plt.ylabel('Time (1/J)')

## 추가 정보

**한글화:** 김경훈, 신소영